*Alina Avanesyan*

### **1. Корпус**

В качестве корпусных данных были взяты стихи, в которых удалось найти следующие группы слов, представляющих собой трудности для автоматического потсттеггинга:

- Омонимы (дали (*сущ.*) - дали (*гл.*), лаем (*сущ., Т.п., ед.ч.*) - лаем (*гл.*)) (сложность в том, что написание слов идентичное, а часть речи можно определить только в контексте)
- Слова с дефисным написанием (имена собственные (*Сант-Яго*), составные числительные (*десяти-двенадцати*))
- Причастия/прилагательные (прошедшего, текущая)
- Слова, придуманные автором (*громадьё*, *разжелудясь*)

In [408]:
#открываем файл с подготовленными текстами и записываем тексты в строку
data = ''
with open('data.txt', mode='r', encoding='UTF-8') as file:
    for line in file.readlines():
        data += line.replace('\n', ' ')

In [409]:
#количество токенов
len(data.split(' '))

621

In [289]:
import pandas as pd

#открываем таблицу со сложными для определения части речи словами
diffic = pd.read_excel('dif_forms.xlsx')
diffic['POS'] = diffic['POS'].apply(lambda x: x[1:-1].split(',') if '[' in x else x)
forms = diffic['Словоформа'].to_list()

В качестве тегсета был выбран набор граммем от pymorphy2. Его удобство в том, что состав тегов хорошо описывает языковые особенности (например, в нем фиксируется существование причастий двух видов - полных и кратких, а также отдельно отмечается деепричастие. То есть глагольные формы не объединяются в один тег), необходимые для предотвращения неправильного теггинга. В нем не фиксируются классы местоимений (существует только местоимение-сущ., а остальные виды уже причисляют к другим граммемам (например, прил.)), однако стоит учитывать, что сложности с определением части речи чаще возникают при работе с глагольными формами, нежели чем с местоименными (поскольку число местоименных формы ограничено, они менее подвержены омонимии).

### **2. Разные POS теггеры** + конвертация тегов к единой системе тегов

#### **2.1. Pymorphy2**

Поскольку у Pymorphy2 нет встроенного токенизатора, воспользуемся токенизатором NLTK:

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('russian'))
from nltk.tokenize import word_tokenize
from string import punctuation
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [291]:
result = nltk.word_tokenize(data)

pymorphy2_tags = {}
for word in list(set(result)):
    if word in forms or word.lower() in forms:
        pymorphy2_tags[word] = morph.parse(word)[0].tag.POS

тут не происходит разделения, т.к. у pymorphy2 нет токенизатора, а nltk не разделяет слова по дефисам

#### **2.2. Mystem**

Для экономии памяти компьютера и ускорения алгоритма встраиваем конвертер в процесс "прогона" текста через теггер. Можно было сначала прогнать текст, затем отдельной функции перевести одни теги в другие, согласно словарю. Однако в случае с Mystem, здесь отсутствуют теги для причастий, деепричастий, кратких/полных форм прилагательных/наречий и др. --> просто словарем в данном случае не обойтись, необходимо придумать алгоритм сложнее.

Заметим, что по грамматической хар-ки слова, выдаваемой Mystem, можно определить форму. Например, в случае ниже слову присваивается тег "VERB" (глагол), однако в разделе verb_form содержится признак "прич,полн", который нам нужен вытащить (аналогично и с остальными категориями, отсутствующими в тегсете Mystem):

In [302]:
from pymystem3 import Mystem
m = Mystem()

ana = m.analyze('бегущий по дороге')
ana[0]

{'analysis': [{'lex': 'бежать',
   'wt': 1,
   'gr': 'V,нп=(непрош,вин,ед,прич,полн,муж,несов,действ,неод|непрош,им,ед,прич,полн,муж,несов,действ)'}],
 'text': 'бегущий'}

In [304]:
#словарь для конвертации (слева - mystem, справа - варианты тегов в pymorphy2_tags
mystem_pos = {
    'A': {'positive': 'ADJF', 'short': 'ADJS', 'compar': 'COMP'},
    'ADV': {'ordinary': 'ADV', 'predicative': 'PRED', 'compar': 'COMP'},
    'S': 'NOUN',
    'V': {'fin': 'VERB', 'inf': 'INFN', 'full_participle': 'PRTF', 'short_participle': 'PRTS', 'adverbial_participle': 'GRND'},
    'PR': 'ADP', #перед
    'INTJ': 'INTJ',
    'PART': 'PART', #лишь
    'NUM': 'NUM', #семь
    'ADVPRO': 'ADV', #здесь
    'APRO': 'PRON', #какой-то
    'SPRO': 'PRON', #ты, некто
    'ANUM': 'ADJ', #второй
    'CONJ': 'CONJ', #и
}

In [305]:
from pymystem3 import Mystem
m = Mystem()

mystem_tags = {}
ana = m.analyze(data)

for word in ana:
    if word['text'] in forms or word['text'].lower() in forms:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            tag = gr.split('=')[0].split(',')[0]
            mystem_tags[word['text']] = tag

            if tag == 'ADV':
                if 'прдк' in gr:
                    tag = mystem_pos['ADV']['predicative']
                    
                elif 'срав' in gr:
                        tag = mystem_pos['ADV']['compar']
                else:
                    tag = mystem_pos['ADV']['ordinary']

            elif tag == 'V':
                if 'инф' in gr:
                    tag = mystem_pos['V']['inf']
                elif 'прич' in gr:
                    if 'полн' in gr:
                        tag = mystem_pos['V']['full_participle']
                    else:
                        tag = mystem_pos['V']['short_participle']
                elif 'деепр' in gr:
                    tag = mystem_pos['V']['adverbial_participle']
                else:
                    tag = mystem_pos['V']['fin']
            
            elif tag == 'A':
                if 'кр' in gr:
                    tag = mystem_pos['A']['short']
                elif 'срав' in gr:
                    tag = mystem_pos['A']['compar']
                else:
                    tag = mystem_pos['A']['positive']

            else:
                if tag in mystem_pos.keys():
                    tag = mystem_pos[tag]           

                
            mystem_tags[word['text']] = tag

Заметим, что еще до этапа определения части речи Mystem определяет границы слова (парсит текст), поэтому слова с дефисами библиотека может поделить на два отдельных слова, например:

In [307]:
f = 'двадцати-тридцати'
m.analyze(f)

[{'analysis': [{'lex': 'двадцать', 'wt': 1, 'gr': 'NUM=(пр|дат|род)'}],
  'text': 'двадцати'},
 {'text': '-'},
 {'analysis': [{'lex': 'тридцать', 'wt': 1, 'gr': 'NUM=(пр|дат|род)'}],
  'text': 'тридцати'},
 {'text': '\n'}]

В ручной заметке я тоже разделила эту форму на две отдельных, однако необходимо проверить совпадение тегов в списке тегов для каждой отдельной части, чтобы в дальнейшем оценивать accuracy:

In [306]:
for f in forms:
    if f in data and len(m.analyze(f)) > 2 and m.analyze(f)[1]['text'] == '-':
        new_tags = []
        for t in m.analyze(f):
            if 'analysis' in t:
                gr = t['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                new_tags.append(pos)
        mystem_tags[f] = new_tags

В случае с pymorphy2 нам не нужно проводить подобные манипуляции, так как парсер от nltk обычно не разделяет слова с дефисами.

#### **2.3. Natasha**

In [312]:
natasha_pos = {
    'ADJ': {'full': 'ADJF', 'short': 'ADJS', 'compar': 'COMP'},
    'ADV': {'positive': 'ADVB', 'compar': 'COMP'},
    'VERB': {'fin': 'VERB', 'inf': 'INFN', 'full_participle': 'PRTF', 'short_participle': 'PRTS', 'adverbial_participle': 'GRND'},
    'NOUN': 'NOUN',
    'PRON': 'NPRO',
    'NUM': 'NUMR',
    'CONJ': 'CONJ',
    'INTJ': 'INTJ',
    'DET': 'ADJF',
    'PART': 'PRCL',
    'ADP': 'PREP',
}

In [343]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doc

natasha_tags = {}
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

doc = Doc(data)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for t in doc.tokens:
    if t.text in forms or t.text.lower() in forms:
        if t.pos == 'VERB':
            if 'Conv' in t.feats.values():
                tag = natasha_pos['VERB']['adverbial_participle']
            elif 'Part' in t.feats.values():
                if 'Short' in t.feats.values():
                    tag = natasha_pos['VERB']['short_participle']
                else:
                    tag = natasha_pos['VERB']['full_participle']
            elif 'Inf' in t.feats.values():
                tag = natasha_pos['VERB']['inf']
        elif t.pos == 'ADJ':
            if 'Short' in t.feats.values():
                tag = natasha_pos['ADJ']['short']
            elif 'Cmp' in t.feats.values():
                tag = natasha_pos['ADJ']['compar']
            else:
                tag = natasha_pos['ADJ']['full']
        elif t.pos == 'ADV':
            if 'Cmp' in t.feats.values():
                tag = natasha_pos['ADV']['compar']
            else:
                tag = natasha_pos['ADV']['positive']
        else:
            if t.pos in natasha_pos.keys():
                tag = natasha_pos[t.pos]           
            else:
                tag = t.pos
        natasha_tags[t.text] = tag

Примечательно, что Natasha очень чувствительный к регистру анализатор. Если слово нарицательное, но первая его буква заглавная, то Natasha определяет его как "PROPN" (имя собственное), что реже наблюдается у других анализаторов (--> можно ожидать более низкое качество):

In [323]:
doc = Doc('Уснув, он забыл обо всем')
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.tokens[0]

DocToken(stop=5, text='Уснув', pos='PROPN', feats=<Anim,Nom,Masc,Sing>)

In [324]:
#здесь анализатор плохо справляется с деепричастием, после которого нет дополнения
doc = Doc('уснув, он забыл обо всем')
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.tokens[0]

DocToken(stop=5, text='уснув', pos='NOUN', feats=<Inan,Nom,Masc,Sing>)

In [328]:
doc = Doc('Увидев')
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.tokens[0]

DocToken(stop=6, text='Увидев', pos='PROPN', feats=<Anim,Nom,Masc,Sing>)

In [325]:
morph.parse('уснув')[0].tag.POS

'GRND'

In [329]:
morph.parse('Увидев')[0].tag.POS

'GRND'

### **3. Accuracy**

In [332]:
def compare(res_tags):
    diffic['Словоформа'] = diffic['Словоформа'].apply(lambda x: x.lower())
    right_tags = dict(zip(diffic.Словоформа, diffic.POS))
    res_tags = {k.lower(): v for k, v in res_tags.items()}
    right = 0
    for key in res_tags.keys():
        if type(right_tags[key]) != list:
            if res_tags[key] == right_tags[key]:
                right += 1
        elif type(res_tags[key.lower()]) == list and type(right_tags[key.lower()]) != list:
            if res_tags[key.lower()] == right_tags[key]:
                right += 1
    return [round(right/len(res_tags.keys()), 3), f'Верно: {right}, Неверно: {len(res_tags.keys())-right}']

In [341]:
#точность pymorphy2
compare(pymorphy2_tags)

[0.737, 'Верно: 14, Неверно: 5']

In [344]:
#точность natasha
compare(natasha_tags)

[0.421, 'Верно: 8, Неверно: 11']

Попробуем проверить точность Наташи после приведения слов к нижнему регистру (чтобы избежать описанную в п. 2.3 проблему):

In [338]:
natasha_tags = {}
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

doc = Doc(data.lower())
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for t in doc.tokens:
    if t.text in forms or t.text.lower() in forms:
        if t.pos == 'VERB':
            if 'Conv' in t.feats.values():
                tag = natasha_pos['VERB']['adverbial_participle']
            elif 'Part' in t.feats.values():
                if 'Short' in t.feats.values():
                    tag = natasha_pos['VERB']['short_participle']
                else:
                    tag = natasha_pos['VERB']['full_participle']
            elif 'Inf' in t.feats.values():
                tag = natasha_pos['VERB']['inf']
        elif t.pos == 'ADJ':
            if 'Short' in t.feats.values():
                tag = natasha_pos['ADJ']['short']
            elif 'Cmp' in t.feats.values():
                tag = natasha_pos['ADJ']['compar']
            else:
                tag = natasha_pos['ADJ']['full']
        elif t.pos == 'ADV':
            if 'Cmp' in t.feats.values():
                tag = natasha_pos['ADV']['compar']
            else:
                tag = natasha_pos['ADV']['positive']
        else:
            if t.pos in natasha_pos.keys():
                tag = natasha_pos[t.pos]           
            else:
                tag = t.pos
        natasha_tags[t.text] = tag

compare(natasha_tags)

[0.5, 'Верно: 9, Неверно: 9']

In [346]:
#точность mystem
compare(mystem_tags)

[0.579, 'Верно: 11, Неверно: 8']

--> модели можно разместить в следующем по убыванию точности порядке:
1) pymorphy2
2) mystem
3) natasha

--> для следующей задачи используем Pymorphy2

### **4. Chuncker**

Скачаем собранные и лемматизированные в предыдущей домашке тексты:

In [355]:
import re
#собранные с сайта отзывы
data_full = pd.read_csv('hw1_data.csv', delimiter=',')
del data_full['Unnamed: 0']
data_full['Rate'] = data_full['Rate'].astype(int)
data_full['Real'] = data_full['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

#лемматизированные данные
data = pd.read_csv('hw1_all_data_tokens.csv', delimiter=',')
del data['Unnamed: 0']
data['Rate'] = data['Rate'].astype(int)
data['Real'] = data['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

#при сохранении файла меняется тип данных в колонке Review, поэтому из строки нужно опять сделать список
def clean_tokens(text):
    text2 = []
    for i in range(len(text)):
        text[i] = re.sub("'|«|»|,| ,|, ,| ", '', text[i])
        if text[i].isalpha():
          text2.append(text[i])
    return text2

data['Review'] = data['Review'].apply(lambda x: x[1:-2].split("',"))
data['Review'] = data['Review'].apply(clean_tokens)
data = data[data['Rate'] != 3]

In [367]:
data_sample = data.sample(frac=0.2)
data_sample.head(5)

,Name,Review,Rate,Real
1343,"Фильм ""Черная вдова"" (2021)","[итак, начать, с, тот, что, данный, фильм, чёр...",4,положительный
1364,"Фильм ""Бык"" (2019)","[приветствовать, весь, читатель, занлянуть, в,...",5,положительный
2224,"Фильм ""Калашников"" (2020)","[биографический, фильм, калашников, решить, по...",2,отрицательный
1780,"Фильм ""Поменяться местами"" (2019)","[добрый, день, сегодня, наткнуться, на, очень,...",5,положительный
762,"Фильм ""Вышка"" (2022)","[весь, привет, в, свой, сегодняшний, отзыв, я,...",4,положительный


- тональный словарь:

In [368]:
positive = data_sample[data_sample['Rate'] >= 4]['Review'].tolist()
negative = data_sample[data_sample['Rate'] < 4]['Review'].tolist()
positive = [element for each_list in positive for element in each_list]
negative = [element for each_list in negative for element in each_list]
only_positive = [x for x in positive if x not in negative]
only_negative = [x for x in negative if x not in positive]

In [369]:
only_positive_freq = []
for word in only_positive:
  if only_positive.count(word) > 2:
    only_positive_freq.append(word)

only_negative_freq = []
for word in only_negative:
  if only_negative.count(word) > 2:
    only_negative_freq.append(word)

only_positive_freq = set(only_positive_freq)
only_negative_freq = set(only_negative_freq)

- подсчет позитивных и негативных слов

In [370]:
def sentiment(text):
  positive_count = int()
  negative_count = int()
  for lemma in text:
    if lemma in only_positive_freq:
      positive_count += 1
    elif lemma in only_negative_freq:
      negative_count += 1
  if positive_count > negative_count:
    return 'положительный'
  elif positive_count < negative_count:
    return 'отрицательный'
  else:
    return 'нейтральный'

In [383]:
data_sample = data.sample(frac=0.2)
data_sample['Estimate'] = data_sample['Review'].apply(sentiment)
#добавим разметку на основе звезд, присужденных каждому отзыву пользователем
data_sample['Real'] = data_sample['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat = {'неверно': 0, 'верно': 0}
for index, row in data_sample.iterrows():
    if row['Estimate'] != row['Real']:
      stat['неверно'] += 1
    else:
      stat['верно'] += 1

print('Верно:', stat['верно'], 'Неверно:', stat['неверно'], 'Процент точности:', round(stat['верно']/(stat['верно']+stat['неверно'])*100, 4))

Верно: 462 Неверно: 68 Процент точности: 87.1698


##### Находим частотные биграммы

In [388]:
from nltk.util import ngrams

pos_bigrams = {}
neg_bigrams = {}

for index, row in data_sample.iterrows():
    bigrams = ngrams(row['Review'], 2)
    if row['Real'] == 'положительный':
        for b in bigrams:
            if b not in pos_bigrams.keys():
                pos_bigrams[b] = 1
            else:
                pos_bigrams[b] += 1
    elif row['Real'] == 'отрицательный':
        for b in bigrams:
            if b not in neg_bigrams.keys():
                neg_bigrams[b] = 1
            else:
                neg_bigrams[b] += 1

def filter_dict(ngram_item):
    key, value = ngram_item
    if value > 4:
        return True
    else:
        return False

pos_bigrams = dict(filter(filter_dict, pos_bigrams.items()))
neg_bigrams = dict(filter(filter_dict, neg_bigrams.items()))

pos_bigrams = dict(sorted(pos_bigrams.items(), key=lambda x: x[1], reverse=True))
neg_bigrams = dict(sorted(neg_bigrams.items(), key=lambda x: x[1], reverse=True))

pos_bigrams_filt = dict([x for x in pos_bigrams.items() if x[0] not in neg_bigrams.keys()])
neg_bigrams_filt = dict([x for x in neg_bigrams.items() if x[0] not in pos_bigrams.keys()])

In [379]:
dict(sorted(neg_bigrams_filt.items(), key=lambda x: x[1], reverse=True))

{('не', 'рекомендовать'): 15,
 ('просмотр', 'не'): 7,
 ('потратить', 'время'): 6,
 ('не', 'понимать'): 6,
 ('то', 'ли'): 6,
 ('снять', 'фильм'): 5,
 ('с', 'весь'): 5,
 ('не', 'смешной'): 5,
 ('и', 'смотреть'): 5}

1) Заметим, что для негативных отзывов характерно сочетание частицы *не* с глаголом (нам необходимо найти как можно больше признаков негативных отзывов, т.к. тональный словарь для отрицательных текстов меньше, выборка несбалансированная (общая тенденция людей писать больше положительные отзывы)) (примеры: мне (не) нравится, я (не) рекомендую).

2) Также можно рассмотреть группы "глагол + наречие", где один и тот же глагол в  сочетании с разными наречиями может давать как положительную, так и отрицательную окраску. Будем сравнивать частоту биграмм (биграммы про положительное, если ее частотность выше в положительных текстах), а если не получится, искать биграмму среди уникальных (создаем тональный словарь из биграмм).

3) Усилительная частица *очень* в сочетании с прилигательным или наречием может давать яркую хар-ку объекту --> служить маркером тональности (проверяем, в каком списке (положительном или отрицательном) чаще всего появляется слово, после присваиваем +1 положительности или отрицательности).

4) Можно проверить сочетания по типу *плохо* + VERB / VERB + *плохо* / *плохой* + NOUN (в зависимости от выборки может случайно отсечь наречие *плохо" во 2ом пункте, поэтому такой очевидный маркер стоит рассмотреть отдельно).

In [405]:
def chunker(text):
    posit = int()
    negat = int()
    for i in range(len(text)-1):
        if morph.parse(text[i])[0].tag.POS == 'INFN' and morph.parse(text[i+1])[0].tag.POS == 'ADVB':
            #INFN, а не VERB, т.к. мы уже рассматриваем леммы
            if (text[i], text[i+1]) in pos_bigrams.keys() and (text[i], text[i+1]) in neg_bigrams.keys():
                if pos_bigrams[(text[i], text[i+1])] > neg_bigrams[(text[i], text[i+1])] and pos_bigrams[(text[i], text[i+1])]-neg_bigrams[(text[i], text[i+1])] > 2:
                    posit += 1
                elif pos_bigrams[(text[i], text[i+1])] < neg_bigrams[(text[i], text[i+1])] and neg_bigrams[(text[i], text[i+1])]-pos_bigrams[(text[i], text[i+1])] > 2:
                    negat += 1
               
        elif (text[i] == 'не' and morph.parse(text[i+1])[0].tag.POS == 'INFN') or (text[i] == 'не' and morph.parse(text[i+1])[0].tag.POS == 'ADJF'):
            if (text[i], text[i+1]) in pos_bigrams.keys() and (text[i], text[i+1]) in neg_bigrams.keys():
                if pos_bigrams[(text[i], text[i+1])] > neg_bigrams[(text[i], text[i+1])] and pos_bigrams[(text[i], text[i+1])]-neg_bigrams[(text[i], text[i+1])] > 2:
                    posit += 1
                elif pos_bigrams[(text[i], text[i+1])] < neg_bigrams[(text[i], text[i+1])] and neg_bigrams[(text[i], text[i+1])]-pos_bigrams[(text[i], text[i+1])] > 2:
                    negat += 1
            elif (text[i], text[i+1]) in pos_bigrams_filt.keys():
                posit += 1
            elif (text[i], text[i+1]) in neg_bigrams_filt.keys():
                negat += 1
        elif text[i] == 'плохо' and morph.parse(text[i])[0].tag.POS == 'INFN':
            negat += 1
        
        elif text[i] == 'плохой' and morph.parse(text[i])[0].tag.POS == 'NOUN':
            negat += 1
    
        elif text[i] == 'очень' and (morph.parse(text[i+1])[0].tag.POS == 'ADJF' or morph.parse(text[i+1])[0].tag.POS == 'ADVB'):
            if (text[i], text[i+1]) in pos_bigrams.keys() and (text[i], text[i+1]) in neg_bigrams.keys():
                if pos_bigrams[(text[i], text[i+1])] > neg_bigrams[(text[i], text[i+1])] and pos_bigrams[(text[i], text[i+1])]-neg_bigrams[(text[i], text[i+1])] > 2:
                    posit += 1
                elif pos_bigrams[(text[i], text[i+1])] < neg_bigrams[(text[i], text[i+1])] and neg_bigrams[(text[i], text[i+1])]-pos_bigrams[(text[i], text[i+1])] > 2:
                    negat += 1
            elif text[i+1] in positive or text[i+1] in negative:
                pos_count = positive.count(text[i+1])
                neg_count = negative.count(text[i+1])
                if pos_count > neg_count and pos_count-neg_count > 5:
                    negat += 1
                elif pos_count < neg_count and neg_count-pos_count > 5:
                    posit += 1
        
    return [posit, negat]

In [381]:
def sentiment(text):
  positive_count = int()
  negative_count = int()
  for lemma in text:
    if lemma in only_positive_freq:
      positive_count += 1
    elif lemma in only_negative_freq:
      negative_count += 1

  bigrams_check = chunker(text)
  positive_count += bigrams_check[0]
  negative_count += bigrams_check[1]
  
  if positive_count > negative_count:
    return 'положительный'
  elif positive_count < negative_count:
    return 'отрицательный'
  else:
    return 'нейтральный'

In [382]:
data_sample = data.sample(frac=0.2)
data_sample['Estimate'] = data_sample['Review'].apply(sentiment)
#добавим разметку на основе звезд, присужденных каждому отзыву пользователем
data_sample['Real'] = data_sample['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat = {'неверно': 0, 'верно': 0}
for index, row in data_sample.iterrows():
    if row['Estimate'] != row['Real']:
      stat['неверно'] += 1
    else:
      stat['верно'] += 1

print('Верно:', stat['верно'], 'Неверно:', stat['неверно'], 'Процент точности:', round(stat['верно']/(stat['верно']+stat['неверно'])*100, 4))

Верно: 478 Неверно: 52 Процент точности: 90.1887


Также я проверяла биграмму *не* + *прилагательное* (среди отрицательных биграмм часто встречается отрицание положительного признака: не смешной момент, не интересный сюжет, и т.д.), где должна была усиливаться отрицательность высказывания, если прилагательное было положительным (положительность определялась кол-вом вхождений в положительный и отрицательный список), однако этот способ либо не приносил результата (существенно не повлиял на accuracy), либо уменьшал точность (так как выборки брались рандомные, то положительное, но редкое слово могло стать более популярным в отрицательных отзывах).

Таким образом, качество модели повысилось на 2-3% за счет добавления возможности учитывать не только слова из тонального словаря, но и биграмм.